# DeepSeek-R1-Distilled-14B

In [1]:
from llama_cpp import Llama
import json
import pandas as pd

In [2]:
path_to_model = '/home/ubuntu/Final_project/DeepSeek-R1-Distill-Qwen-14B-Q5_K_M.gguf'
llm = Llama(model_path = path_to_model, 
            n_ctx = 9999, 
            max_new_tokens = 2048, 
            temperature = 0.1, 
            cache = False, 
            verbose = False,
            n_gpu_layers = -1,
            n_threads = 6,
            chat_format = 'chatml')


llama_context: n_ctx_per_seq (9999) < n_ctx_train (131072) -- the full capacity of the model will not be utilized


In [11]:
label_format = {
    "type": "json_object",
    "schema": {
        "type": "object",
        "properties": {
            "imatinib_mentioned": {
                "type": "boolean",
                "description": "True if the drug imatinib is mentioned in the note"
            },
            "related_drugs_mentioned": {
                "type": "boolean",
                "description": "True if drugs related to imatinib (e.g., dasatinib, nilotinib, bosutinib) are mentioned"
            },
            "cml_diagnosed": {
                "type": "boolean",
                "description": "True if chronic myeloid leukemia (CML) is diagnosed"
            },
            "cml_in_regression": {
                "type": "boolean",
                "description": "True if chronic myeloid leukemia is mentioned as being in regression"
            },
            "aml_diagnosed": {
                "type": "boolean",
                "description": "True if acute myeloid leukemia (AML) is diagnosed"
            },
            "blast_phase_cml": {
                "type": "boolean",
                "description": "True if blast phase CML is explicitly mentioned"
            },
            "bmt_history": {
                "type": "boolean",
                "description": "True if there is mention of history of bone marrow transplant (BMT)"
            },
            "acute_phase_cml": {
                "type": "boolean",
                "description": "True if acute phase CML is explicitly mentioned"
            }
        },
        "required": [
            "imatinib_mentioned",
            "related_drugs_mentioned",
            "cml_diagnosed",
            "cml_in_regression",
            "aml_diagnosed",
            "blast_phase_cml",
            "bmt_history",
            "acute_phase_cml"
        ]
    }
}


In [12]:
system_prompt = """
You are a strict JSON generator. Only output JSON that matches the schema below.
Do not include any extra text, commentary, or explanations.

Schema:
- imatinib_mentioned: true if the drug imatinib (also known as Gleevec) is mentioned in the note, otherwise false
- related_drugs_mentioned: true if drugs related to imatinib (e.g., dasatinib, nilotinib, bosutinib) are mentioned, otherwise false
- cml_diagnosed: true if chronic myeloid leukemia (CML) is diagnosed, otherwise false
- cml_in_regression: true if chronic myeloid leukemia is mentioned as being in regression, otherwise false
- aml_diagnosed: true if acute myeloid leukemia (AML) is diagnosed, otherwise false
- blast_phase_cml: true if blast phase CML is explicitly mentioned, otherwise false
- bmt_history: true if history of bone marrow transplant (BMT) is mentioned, otherwise false
- acute_phase_cml: true if acute phase CML is explicitly mentioned, otherwise false

Rules:
1. Only mark a field as true if the note clearly indicates it.
2. If the note does not explicitly mention a field, mark it false.
3. The output must always be valid JSON with all eight fields present.
"""


In [5]:
# sample_notes = clinical_notes_all_path = "latest_notes.parquet"
# clinical_notes_all = pd.read_parquet(clinical_notes_all_path, engine='pyarrow')
# clinical_notes_all.columns
# clinical_notes_all['note_text']
# first_300 = clinical_notes_all['note_text']
# first_300


# Load parquet into pandas df

In [8]:
df = pd.read_parquet('latest_notes.parquet')
#df = df.iloc[:40]
sample_notes = df['note_text'].values.tolist()


# Execute inference

In [13]:
llm_results = []

for i, note in enumerate(sample_notes):
    structured_output = llm.create_chat_completion(
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": note}
        ],
        response_format=label_format,
        temperature=0.3,
    )

    results = json.loads(structured_output['choices'][0]['message']['content'])

    llm_results.append({
        "note_text": note,
        "llm_imatinib_mentioned": results.get('imatinib_mentioned'),
        "llm_related_drugs_mentioned": results.get('related_drugs_mentioned'),
        "llm_cml_diagnosed": results.get('cml_diagnosed'),
        "llm_cml_in_regression": results.get('cml_in_regression'),
        "llm_aml_diagnosed": results.get('aml_diagnosed'),
        "llm_blast_phase_cml": results.get('blast_phase_cml'),
        "llm_bmt_history": results.get('bmt_history'),
        "llm_acute_phase_cml": results.get('acute_phase_cml')
    })

    if i % 10 == 0:
        print(f"Processed {i} / {len(sample_notes)} notes")


Processed 0 / 1762 notes
Processed 10 / 1762 notes
Processed 20 / 1762 notes
Processed 30 / 1762 notes
Processed 40 / 1762 notes
Processed 50 / 1762 notes
Processed 60 / 1762 notes
Processed 70 / 1762 notes
Processed 80 / 1762 notes
Processed 90 / 1762 notes
Processed 100 / 1762 notes
Processed 110 / 1762 notes
Processed 120 / 1762 notes
Processed 130 / 1762 notes
Processed 140 / 1762 notes
Processed 150 / 1762 notes
Processed 160 / 1762 notes
Processed 170 / 1762 notes
Processed 180 / 1762 notes
Processed 190 / 1762 notes
Processed 200 / 1762 notes
Processed 210 / 1762 notes
Processed 220 / 1762 notes
Processed 230 / 1762 notes
Processed 240 / 1762 notes
Processed 250 / 1762 notes
Processed 260 / 1762 notes
Processed 270 / 1762 notes
Processed 280 / 1762 notes
Processed 290 / 1762 notes
Processed 300 / 1762 notes
Processed 310 / 1762 notes
Processed 320 / 1762 notes
Processed 330 / 1762 notes
Processed 340 / 1762 notes
Processed 350 / 1762 notes
Processed 360 / 1762 notes
Processed 37

In [14]:
with open("DeepSeek_R1_14b_Output_additonal_prompt.json", "w") as f:
    json.dump(llm_results, f, indent=2)